In [1]:
from pyspark import SparkContext
sc=SparkContext("local","Pyspark new")

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
print("Spark Version:", spark.version)

Spark Version: 3.5.6


In [3]:
clickstream_df = spark.read \
    .option("delimiter", "\t") \
    .option("header", False) \
    .option("inferSchema", True) \
    .csv("clickstream-enwiki-2025-07.tsv")


In [4]:
columns = ["prev", "current", "type", "n"]
clickstream_df = clickstream_df.toDF(*columns)  # writing the column headers

# show few rows
clickstream_df.show(10)

+-------------------+--------------------+--------+---------+
|               prev|             current|    type|        n|
+-------------------+--------------------+--------+---------+
|        other-empty|           Main_Page|external|167194874|
|Meteor_(miniseries)|     Ernie_Barbarash|    link|       13|
|       other-search|       Ozzy_Osbourne|external|  7956732|
|   Meteor_(missile)|                Euro|    link|       13|
|        other-empty|        Hyphen-minus|external|  7819580|
|   Meteor_(missile)|      Royal_Ordnance|    link|       13|
|     other-internal|           Main_Page|external|  6902577|
|   Meteor_(missile)|Saab_Bofors_Dynamics|    link|       13|
|       other-search|Superman_(2025_film)|external|  5462456|
|   Meteor_(missile)|     Sukhoi_Su-30MKI|    link|       13|
+-------------------+--------------------+--------+---------+
only showing top 10 rows



In [5]:
# check the schema
clickstream_df.printSchema()

root
 |-- prev: string (nullable = true)
 |-- current: string (nullable = true)
 |-- type: string (nullable = true)
 |-- n: integer (nullable = true)



In [6]:
clickstream_df.columns

['prev', 'current', 'type', 'n']

In [7]:
noOfRows= clickstream_df.count()

In [8]:
noOfRows

36131419

Prajwal Part 

Schema exploration

In [9]:
clickstream_df.printSchema()
clickstream_df.show(5, truncate=False)

root
 |-- prev: string (nullable = true)
 |-- current: string (nullable = true)
 |-- type: string (nullable = true)
 |-- n: integer (nullable = true)

+-------------------+---------------+--------+---------+
|prev               |current        |type    |n        |
+-------------------+---------------+--------+---------+
|other-empty        |Main_Page      |external|167194874|
|Meteor_(miniseries)|Ernie_Barbarash|link    |13       |
|other-search       |Ozzy_Osbourne  |external|7956732  |
|Meteor_(missile)   |Euro           |link    |13       |
|other-empty        |Hyphen-minus   |external|7819580  |
+-------------------+---------------+--------+---------+
only showing top 5 rows



Missing values check

In [10]:
from pyspark.sql import functions as F

missing_counts = clickstream_df.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in clickstream_df.columns
])
missing_counts.show()


# No Missing Values

+----+-------+----+---+
|prev|current|type|  n|
+----+-------+----+---+
|   0|      0|   0|  0|
+----+-------+----+---+



Statistics (mean, min, max of n)

In [11]:
from pyspark.sql import functions as F   # <--- missing import

stats = clickstream_df.select(
    F.min("n").alias("min_n"),
    F.max("n").alias("max_n"),
    F.mean("n").alias("mean_n")
)

stats.show()

+-----+---------+------------------+
|min_n|    max_n|            mean_n|
+-----+---------+------------------+
|   10|167194874|181.54733773395392|
+-----+---------+------------------+



Quantiles (median, etc.)

In [12]:
quantiles = clickstream_df.approxQuantile("n", [0.25, 0.5, 0.75, 0.9, 0.99], 0.01)
print("Quantiles:", quantiles)


Quantiles: [15.0, 26.0, 63.0, 187.0, 167194874.0]


Sampling for quick exploration

In [13]:
sample_df = clickstream_df.sample(fraction=0.0001, seed=42)  # 0.01% sample ~3.6k rows
sample_df.show(10, truncate=False)


+-----------------------------------+---------------------------+--------+-----+
|prev                               |current                    |type    |n    |
+-----------------------------------+---------------------------+--------+-----+
|other-search                       |Mikel_Merino               |external|21499|
|Murders_of_Blake_and_Mary-Jo_Hadley|House_party                |link    |13   |
|My_Love:_Essential_Collection      |Dance_with_My_Father_(song)|link    |13   |
|Myki                               |Go_card                    |link    |13   |
|Mickey_Hargitay                    |Jayne_Mansfield            |link    |17224|
|Napoleon_II                        |Vienna                     |link    |13   |
|Narcissistic_defences              |History_of_narcissism      |link    |13   |
|other-search                       |Marc_Pubill                |external|15166|
|Nirakkoottu                        |Urvashi_(actress)          |link    |13   |
|other-search               

Validate n column  ( Lecture - 3 )

In [14]:
from pyspark.sql import functions as F

# Find rows where n is not a pure number
df_invalid_n = clickstream_df.filter(~F.col("n").cast("string").rlike("^[0-9]+$"))
df_invalid_n.show(20, truncate=False)

# Count them
print("Invalid n rows:", df_invalid_n.count())



# So the Data Set is clean   ( As from the output )

+----+-------+----+---+
|prev|current|type|n  |
+----+-------+----+---+
+----+-------+----+---+

Invalid n rows: 0


Inspect type values

In [15]:
clickstream_df.groupBy("type").count().show(truncate=False)


# Output : No Flag is found 

+--------+--------+
|type    |count   |
+--------+--------+
|link    |22940461|
|other   |790412  |
|external|12400546|
+--------+--------+



Inspect prev column

In [16]:
clickstream_df.groupBy("prev").count().orderBy(F.desc("count")).show(20, truncate=False)        


+-------------------------------------------------------------+-------+
|prev                                                         |count  |
+-------------------------------------------------------------+-------+
|other-empty                                                  |5696646|
|other-search                                                 |4075166|
|other-internal                                               |1785075|
|other-external                                               |642457 |
|Main_Page                                                    |263151 |
|other-other                                                  |201202 |
|Deaths_in_2025                                               |5616   |
|Wikipedia                                                    |3684   |
|Wiki                                                         |2743   |
|List_of_United_States_counties_and_county_equivalents        |2480   |
|List_of_American_films_of_2025                               |2

Inspect curr column

In [17]:
clickstream_df.groupBy("current").count().orderBy(F.desc("count")).show(20, truncate=False)                                                             


+--------------------------------+------+
|current                         |count |
+--------------------------------+------+
|Main_Page                       |112581|
|United_States                   |5438  |
|YouTube                         |3420  |
|World_War_II                    |2763  |
|United_Kingdom                  |2701  |
|Donald_Trump                    |2623  |
|New_York_City                   |2515  |
|Superman_(2025_film)            |2497  |
|India                           |2399  |
|Order_of_the_British_Empire     |2295  |
|China                           |2264  |
|Catholic_Church                 |2120  |
|Academy_Awards                  |2093  |
|Los_Angeles                     |2022  |
|Russia                          |1962  |
|Soviet_Union                    |1941  |
|Democratic_Party_(United_States)|1831  |
|Ottoman_Empire                  |1736  |
|Netflix                         |1717  |
|Japan                           |1715  |
+--------------------------------+